In [ ]:
##########################################Thesis_Implementation################################################################

### Importing Required Libraries  

In [1]:
#later add the libraries with links and what each library does just as an information 
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve,auc
from sklearn.cluster import KMeans
from nltk.stem.porter import PorterStemmer 

import re 
import string 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from nltk.stem.wordnet import WordNetLemmatizer 
import pickle

### Mythril Disassembly code used to transpate bytecodes to opcodes 

In [2]:
#2.about the disassembler 
#using the mythril python library for disassembly 
#https://github.com/ConsenSys/mythril
#takes bytecode and ouputs machine level opcodes 
#write basic working of the disassembler code later

from mythril.ether import asm,util
import os
import json
import logging


class Disassembly:

    def __init__(self, code):
        self.instruction_list = asm.disassemble(util.safe_decode(code))
        self.xrefs = []
        self.func_to_addr = {}
        self.addr_to_func = {}

        try:
            mythril_dir = os.environ['MYTHRIL_DIR']
        except KeyError:
            mythril_dir = os.path.join(os.path.expanduser('~'), ".mythril")

        # Load function signatures

        signatures_file = os.path.join(mythril_dir, 'signatures.json')

        if not os.path.exists(signatures_file):
            logging.info("Missing function signature file. Resolving of function names disabled.")
            signatures = {}
        else:
            with open(signatures_file) as f:
                signatures = json.load(f)

        # Parse jump table & resolve function names

        jmptable_indices = asm.find_opcode_sequence(["PUSH4", "EQ"], self.instruction_list)

        for i in jmptable_indices:
            func_hash = self.instruction_list[i]['argument']
            try:
                func_name = signatures[func_hash]
            except KeyError:
                func_name = "_function_" + func_hash

            try:
                offset = self.instruction_list[i+2]['argument']
                jump_target = int(offset, 16)

                self.func_to_addr[func_name] = jump_target
                self.addr_to_func[jump_target] = func_name
            except:
                continue



    def get_easm(self):

        return asm.instruction_list_to_easm(self.instruction_list)

### EVM opcodes as provides in the Ethereum Yellow Paper 

In [3]:
#3.EVM opcodes 
#source Ethereum yellow paper 
#Document what each opcode is doing 
EVM_OPCODE_SET = {'STOP', 'ADD', 'MUL', 'SUB', 'DIV', 'SDIV', 'MOD', 'SMOD', 'ADDMOD', 'MULMOD', 'EXP', 'SIGNEXTEND',
                 'LT','GT', 'SLT', 'SGT', 'EQ', 'ISZERO', 'AND', 'OR', 'XOR', 'NOT', 'BYTE','SHA3',
                 'ADDRESS', 'BALANCE', 'ORIGIN', 'CALLER', 'CALLVALUE', 'CALLDATALOAD', 'CALLDATASIZE', 'CALLDATACOPY', 
                  'CODESIZE', 'CODECOPY', 'GASPRICE','EXTCODESIZE','EXTCODECOPY', 'RETURNDATASIZE', 'RETURNDATACOPY',
                 'BLOCKHASH', 'COINBASE', 'TIMESTAMP', 'NUMBER', 'DIFFICULTY', 'GASLIMIT','POP','MLOAD', 'MSTORE', 'MSTORE8'
                 'SLOAD', 'SSTORE', 'JUMP', 'JUMPI', 'PC', 'MSIZE', 'GAS', 'JUMPDEST','PUSH1','PUSH2','PUSH3','PUSH4',
                  'PUSH5','PUSH6','PUSH7','PUSH8','PUSH9','PUSH10','PUSH11','PUSH12','PUSH13','PUSH14','PUSH15','PUSH16',
                 'PUSH17','PUSH18','PUSH19','PUSH20','PUSH21','PUSH22','PUSH23','PUSH24','PUSH25','PUSH26','PUSH27',
                 'PUSH28','PUSH29','PUSH30','PUSH31','PUSH32','DUP1','DUP2','DUP3','DUP4','DUP5','DUP6','DUP7','DUP8','DUP9',
                 'DUP10','DUP11','DUP12','DUP13','DUP14','DUP15','DUP16','DUP17','DUP18','SWAP1','SWAP2','SWAP3','SWAP4',
                 'SWAP5','SWAP6','SWAP7','SWAP8','SWAP9','SWAP10','SWAP11','SWAP12','SWAP13','SWAP14','SWAP15','SWAP16',
                 'LOG0','LOG1','LOG2','LOG3','LOG4','CREATE', 'CAL', 'CALLCODE','RETURN','DELEGATECALL' ,'STATICCALL','REVERT', 
                  'INVALID','SELFDESTRUCT'}

### Loading data, Translating bytecodes to opcodes, Processing opcodes and get rid of arguments 

In [ ]:
#loading data
#df = pd.read_csv('/Users/ravishaggarwal/Desktop/contracts.csv',sep = ',')
df = pd.read_csv("/Users/ravishaggarwal/Desktop/thesis_impl/implementations/data/toy_data/new_data1.csv",sep =',')

In [ ]:
#bytecodes that are to be translated to opcodes 
bytecode = df['result.code']

In [ ]:
#getting the list of opcodes translated from bytecodes 
opcode = (list(map(lambda x: Disassembly(x).get_easm(),bytecode)))

In [ ]:
#converting opcodes list to dataframe
opcode_df = pd.DataFrame(opcode)

In [ ]:
#naming the opcodes dataframe
opcode_df.columns = ['opcode']

In [ ]:
#adding the opcodes data frame into the original data frame 
df['opcode'] = opcode_df

In [ ]:
#selecting the bytecode and the translated opcodes 
new_df = df[df.columns[3:]]
#new_df = df.columns['result.code','opcode']

In [ ]:
#new_df is the new dataframe consist of translated opcodes 
#new_df.head(5)

In [ ]:
#################################################CSV_WRITE############################################################
#writing the result into the csv file for future usage
#new_df.to_csv('new_data.csv', encoding='utf-8', index=False)

In [ ]:
#clean the text from the punctuations 
def cleanpunc(sentence):
    cleaned = re.sub(r'[?|!|\'|"|#|{|}]',r'',str(sentence))
    cleaned = re.sub(r'[.|,|)|(|\|/]',r'',cleaned)
    return cleaned

In [ ]:
#obtaining clean opcodes 
#set of opcodes without arguments are output as a CSV file with the name clean_ops.csv
import sys
import csv

csv.field_size_limit(sys.maxsize)
with open('/Users/ravishaggarwal/Desktop/new_data.csv') as input_file, open('/Users/ravishaggarwal/Desktop/clean_ops.csv', 'w') as output_file:
    r = csv.reader(input_file)
    row = next(r)
    counter = 0
    str1 = ''
    i = 0
    final_string = []
    for row in r:
        var1 = row[1].split()
        #print(var1)
        #counter = counter + 1
        #print(counter)
        #print(var1)
        a = set(var1)&EVM_OPCODE_SET
        #b = list(a)
        
        #print(b)
        #print(a,file = output_file)
        
                    
                    
        
                
            
            #print(sent)
            
        c = cleanpunc(a)
        print(''.join(c),file = output_file)
        
        #print(a)
        #print(b,file = output_file)
        #output_file.write(a)
        #counter = counter + 1
        #print(counter)
        
            
        #for i in var1:
            #print(i)
            #if(i in EVM_OPCODE_SET):
                #output_file.write(i)

In [ ]:
#checking rows in csv 
#with open('/Users/ravishaggarwal/Desktop/new_data.csv') as file:
#    r = csv.reader(file)
#    row_count = sum(1 for row in r)

### Obtaining the Bigram Sparse matrix 

In [ ]:
#bigrams matrix
#bigram sparse matrix is output as a csv file with the name bigrams.csv
import csv
import nltk
from collections import Counter
fo = open("/Users/ravishaggarwal/Desktop/clean_ops.csv")
fo1 = fo.readlines()
counter_sum = Counter()

for line in fo1:
    
    
    #tokens = nltk.word_tokenize(line)
    
    
    bigrams = list(nltk.bigrams(line.split()))  #list of all the possible bigrams 
    #print(bigrams)
    #print(len(bigrams))
    #fdist = nltk.FreqDist(bigrams)
    #for k,v in fdist.items():
        #print (k,v)
    
    
    bigramsC = Counter(bigrams)
    
    
    #counting the no of bigrams combination in each contract 
    #print(bigramsC)
    
    
    #tokensC = Counter(tokens)
    #both_counters = bigramsC + tokensC
    counter_sum = bigramsC
    #print(counter_sum)
    #print(counter_sum)
    #labels, values = zip(*counter_sum.items())
    #print(values)

    #print(tokensC)
    
   
    
with open('/Users/ravishaggarwal/Desktop/#bigrams.csv', 'w', newline='') as csvfile:
    header = sorted(counter_sum, key=lambda x: str(type(x)))
    #print(header)
    writer = csv.DictWriter(csvfile, fieldnames=header)
    writer.writeheader()
    for line in fo1:
          #tokens = nltk.word_tokenize(line)
          bigrams = list(nltk.bigrams(line.split()))
          bigramsC = Counter(bigrams)
          
          #tokensC = Counter(tokens)
          #both_counters = bigramsC + tokensC
          cs = dict(counter_sum)
          bc = dict(bigramsC)
          row = {}
          for element in list(cs):
                if element in list(bc):
                  row[element] = bc[element]
                else:
                  row[element] = 0
          writer.writerow(row)


In [ ]:
'''
#here the idea is to read the bigrams.csv and so column vise sum of all the header as header is our bigram unique vocab
import csv
from __future__ import print_function

sep = ','
with open('/Users/ravishaggarwal/Desktop/bigrams.csv', 'rt') as f:
    #columns = f.readline().strip().split(sep)
    reader = csv.reader(f)
    col = next(reader)
    #print(len(columns))
    rows = [0] * len(col)
    for line in f.readlines():
        data = line.strip().split(sep)
        for i, cell in enumerate(data, start=0):
            rows[i] += float(cell)

print(col)
print(rows)
print(len(col))
print(len(rows))

labels_bigram = col 
value_bigram  = rows
'''    

In [ ]:
#checking out the dimension of bigrams.csv matrix csv 

In [ ]:
import csv
with open("/Users/ravishaggarwal/Desktop/bigrams.csv", "r") as f:
    reader = csv.DictReader(f)
    fieldnames = reader.fieldnames
    row_count = sum(1 for row in reader)

In [ ]:
len(fieldnames)

In [ ]:
row_count

In [ ]:
#printing the top three rows of the bigrams.csv 
import csv
afile = open("/Users/ravishaggarwal/Desktop/bigrams.csv", 'r+')
csvReader1 = csv.reader(afile)
for i in range(3):
    print (csvReader1.__next__())

### Consider the opcodes (Bag Of Words) interpretations 
### Calculating the frequency distributions 

In [ ]:
'''
opcodes = ['CALLDATACOPY', 'PUSH1', 'GAS', 'PUSH20', 'PUSH2', 'CALLVALUE', 'SUB', 'DUP1', 'CALLCODE', 'RETURN', 'CALLDATASIZE']
no of opcodes = 11 
bigrams = [('CALLDATACOPY', 'PUSH1'), ('PUSH1', 'GAS'), ('GAS', 'PUSH20'), ('PUSH20', 'PUSH2'), ('PUSH2', 'CALLVALUE'), ('CALLVALUE', 'SUB'), ('SUB', 'DUP1'), ('DUP1', 'CALLCODE'), ('CALLCODE', 'RETURN'), ('RETURN', 'CALLDATASIZE')]
no of bigrams = 10 


'''

In [ ]:
data = pd.read_csv("/Users/ravishaggarwal/Desktop/c_ops.txt", header=None)

In [ ]:
# import the dataset
from wordcloud import WordCloud
from nltk.corpus import inaugural
# extract the datataset in raw format, you can also extract it in other formats as well
text = str(data)
wordcloud = WordCloud(max_font_size=20).generate(text)
plt.figure(figsize=(16,12))
# plot wordcloud in matplotlib
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
from collections import Counter

with open("/Users/ravishaggarwal/Desktop/c_ops.txt") as f:
    wordcount = Counter(f.read().split())
    #print(wordcount)


In [ ]:
labels, values = zip(*wordcount.items())

In [ ]:
#out of 136 opcodes in the EVM_OPCODES_SET 131 are repeated frequently in the data 
len(labels)


In [ ]:
# sort your values in descending order
indSort = np.argsort(values)[::-1]


In [ ]:
# rearrange your data
labels1 = np.array(labels)[indSort]
values1 = np.array(values)[indSort]
indexes1 = np.arange(len(labels))

In [ ]:
l = labels1[:20]

In [ ]:
l

In [ ]:
v = values1[:20]

In [ ]:
v

In [ ]:
#histogram top 20 values 
#labels, values = zip(*wordcount.items())

#idx = np.arange(len(labels1))
plt.figure(figsize=(25,25))
plt.bar(l, v,width=.2)
plt.show()

In [ ]:
#bottom 20

In [ ]:
l1 = labels1[-20:]

In [ ]:
l1

In [ ]:
v1 = values1[-20:]

In [ ]:
v1

In [ ]:
#histogram bottom 20 values 
#labels, values = zip(*wordcount.items())

#idx = np.arange(len(labels1))
plt.figure(figsize=(25,25))
plt.bar(l1, v1,width=.2)
plt.show()

In [ ]:
#frequency distribution 
with open("/Users/ravishaggarwal/Desktop/c_ops.txt") as f:
    words = [word for word in f.read().split()]
    freqdist = nltk.FreqDist(words)
    plt.figure(figsize=(30,10))
    freqdist.plot()

### Consider the opcodes (Bigram interpretations
### Calculating the frequency distributions

In [ ]:
#clean the text from the punctuations 
def cleanpunc1(sentence):
    cleaned = re.sub(r'[?|!||[|]]',r'',str(sentence))
    cleaned = re.sub(r'[.|||[|]|]',r'',cleaned)
    return cleaned

In [ ]:
#geting bigrams output as bi_gram.csv
import sys
import csv

csv.field_size_limit(sys.maxsize)
with open('/Users/ravishaggarwal/Desktop/clean_ops.csv') as input_file, open('/Users/ravishaggarwal/Desktop/bi_gram1.csv', 'w') as output_file:
    r = csv.reader(input_file)
    row = next(r)
    counter = 0
    str1 = ''
    i = 0
    final_string = []
    for row in r:
        var1 = row[0].split()
        
        #print(var1)
        bigrams = list(nltk.bigrams(var1))
        
        #print(bigrams)
        
        
        #print(var1)
        #counter = counter + 1
        #print(counter)
        #print(var1)
        #a = set(var1)&EVM_OPCODE_SET
        #b = list(a)
        
        #print(b)
        #print(a,file = output_file)
        
                    
                    
        
                
            
            #print(sent)
            
        c = cleanpunc1(bigrams)
        #print(c.rstrip('\n'))
        print(c,file = output_file)
        #print(c.rstrip('\n'),file = output_file)
        #print(a)
        #print(b,file = output_file)
        #output_file.write(a)
        #counter = counter + 1
        #print(counter)
        
            
        #for i in var1:
            #print(i)
            #if(i in EVM_OPCODE_SET):
                #output_file.write(i)

In [ ]:
#here the idea is to read the bigrams.csv and so column vise sum of all the header as header is our bigram unique vocab
import csv
from __future__ import print_function

sep = ','
with open('/Users/ravishaggarwal/Desktop/bigrams.csv', 'rt') as f:
    #columns = f.readline().strip().split(sep)
    reader = csv.reader(f)
    col = next(reader)
    #print(len(columns))
    rows = [0] * len(col)
    for line in f.readlines():
        data = line.strip().split(sep)
        for i, cell in enumerate(data, start=0):
            rows[i] += float(cell)

print(col)
print(rows)
print(len(col))
print(len(rows))

labels_bigram = col 
value_bigram  = rows

In [ ]:
#value_bigram 
# sort your values in descending order
indSort = np.argsort(value_bigram)[::-1]

In [ ]:
#labels_bigram
# rearrange your data
labels_bigram1 = np.array(labels_bigram)[indSort]
values_bigram1 = np.array(value_bigram)[indSort]
indexes_bigram1 = np.arange(len(labels_bigram))

In [ ]:
l_bigram = labels_bigram1[:20]

In [ ]:
l_bigram

In [ ]:
v_bigram = values_bigram1[:20]

In [ ]:
v_bigram

In [ ]:
#labels, values = zip(*wordcount.items())

#idx = np.arange(len(labels1))
plt.figure(figsize=(57,57))
plt.bar(l_bigram, v_bigram,width=.2)
plt.show()

In [ ]:
l_bigram1 = labels_bigram1[-500:]


In [ ]:
l_bigram1

In [ ]:
v_bigram1 = values_bigram1[-200:]

In [ ]:
v_bigram1

In [ ]:
#labels, values = zip(*wordcount.items())

#idx = np.arange(len(labels1))
plt.figure(figsize=(57,57))
plt.bar(l_bigram1, v_bigram1,width=.2)
plt.show()

In [ ]:
#labels, values = zip(*wordcount.items())

#idx = np.arange(len(labels1))
plt.figure(figsize=(50,50))
plt.bar(labels_bigram, value_bigram,width=2)
plt.show()

### Contract Frequency 

In [ ]:
from collections import Counter

with open('/Users/ravishaggarwal/Desktop/bi_gram.txt') as f:
    c = Counter(f)
    dups = [t for t in c.most_common() if t[1] > 1]
    dups_dict = {row: count for row, count in c.most_common() if count > 1}

In [ ]:
key, value = zip(*dups_dict.items())

In [ ]:
key

In [ ]:
len(key)

In [ ]:
value

In [ ]:
len(value)

In [ ]:
value[-8000:]

In [ ]:
#labels, values = zip(*wordcount.items())

#idx = np.arange(len(labels1))
a = [1,2,3,4,5,6,7,8,9,10]
b = [1017390,870791,579340,376008,344381,212472,158860,122783,89134,78287]
plt.figure(figsize=(50,50))
plt.bar(a,b,width= .2)
plt.show()

### Dimensionality Reduction

In [10]:
from sklearn.decomposition import IncrementalPCA
import csv
import sys
import numpy as np
import pandas as pd



dataset = '/Users/ravishaggarwal/Desktop/bigrams.csv'
chunksize_ = 10000
dimensions = 1457

reader = pd.read_csv(dataset, sep = ',', chunksize = chunksize_)


sklearn_pca = IncrementalPCA(n_components=dimensions)

for chunk in reader:
    #print('chunkchunkchunkchunkchunkchunkchunkchunkchunkchunkchunkchunk')
    #print(chunk)
    #y = chunk.pop("Y")
    #print(chunk)
    #print('shape of chunk',chunk)
    sklearn_pca.partial_fit(chunk)
    print('chunking and printing')
    Xchunk = sklearn_pca.transform(chunk)
    
    db = DBSCAN(eps=0.05, min_samples=100).fit_predict(Xchunk[:200])
    
    print(db)
    break
    '''
    #print(Xchunk)
    #reduced_dim = Xchunk[]
    #print('shape of  reduced_dim',reduced_dim.shape)
    #print(.shape)
    a = sklearn_pca.explained_variance_
    
    print(a)
    #print('pca variance explained',a)
    print('shape of pca variance explained',a.shape)
    percentage_var_explained = sklearn_pca.explained_variance_ / np.sum(sklearn_pca.explained_variance_)
    print('shape of percentage_var_explained',percentage_var_explained.shape)
    print('percentage variance explained',percentage_var_explained)
    cum_var_explained = np.cumsum(percentage_var_explained)
    print('cum_var_explained',cum_var_explained)
    print('shape of cum_var_explained',cum_var_explained.shape)
    # Plot the PCA spectrum
    plt.figure(1, figsize=(10, 14))
    plt.clf()
    plt.plot(cum_var_explained, linewidth=2)
    plt.axis('tight')
    plt.axis([0,250,0,1])
    plt.grid()
    plt.xlabel('n_components')
    plt.ylabel('Cumulative_explained_variance')
    plt.show()
    
    
        

    


    

    
    
    #Xtransformed = np.vstack((Xchunk))
    #with open('/Users/ravishaggarwal/Desktop/pca.csv','w') as outfile:
    #    print((Xtransformed ),file = outfile)
        
    

#Computed mean per feature
#mean = sklearn_pca.mean_
#print(mean)
# and stddev
#stddev = np.sqrt(sklearn_pca.var_)
#print(stddev)

Xtransformed = None
for chunk in pd.read_csv(dataset, sep = ',', chunksize = chunksize_):
    #y = chunk.pop("Y")
    Xchunk = sklearn_pca.transform(chunk)
    #print(Xchunk)
    if (Xtransformed is None):
        Xtransformed = Xchunk
    else:
        Xtransformed = np.vstack((Xtransformed, Xchunk))
        print(Xtransformed)
     
        #np.savetxt('test.txt', Xtransformed, delimiter=',')
        
        #with open('/Users/ravishaggarwal/Desktop/pca.csv','w') as outfile:
            #print((Xtransformed),file = outfile)
    #else:
'''       

chunking and printing
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]


In [ ]:
dataset = '/Users/ravishaggarwal/Desktop/bigrams.csv'

db = DBSCAN(eps=0.05, min_samples=100).fit(X)

In [ ]:
#from sklearn.decomposition import IncrementalPCA
import csv
import sys
import numpy as np
import pandas as pd
from sklearn.decomposition import TruncatedSVD



dataset = '/Users/ravishaggarwal/Desktop/bigrams.csv'
chunksize_ = 10000
dimensions = 1456

reader = pd.read_csv(dataset, sep = ',', chunksize = chunksize_)

# Create a TSVD
tsvd = TruncatedSVD(n_components= dimensions)
#sklearn_pca = IncrementalPCA(n_components=dimensions)

for chunk in reader:
    #print('chunkchunkchunkchunkchunkchunkchunkchunkchunkchunkchunkchunk')
    #print(chunk)
    #y = chunk.pop("Y")
    print('chunking and printing')
    Xchunk = tsvd.fit(chunk).transform(chunk)
    #sklearn_pca.partial_fit(chunk)
    
    #Xchunk = sklearn_pca.transform(chunk)
    print(Xchunk)
    percentage_var_explained = tsvd.explained_variance_ / np.sum(tsvd.explained_variance_)
    
    cum_var_explained = np.cumsum(percentage_var_explained)
    # Plot the PCA spectrum
    plt.figure(1, figsize=(6, 4))
    plt.clf()
    plt.plot(cum_var_explained, linewidth=2)
    plt.axis('tight')
    plt.grid()
    plt.xlabel('n_components')
    plt.ylabel('Cumulative_explained_variance')
    plt.show()
    
    
        

    


    

    
    
    #Xtransformed = np.vstack((Xchunk))
    #with open('/Users/ravishaggarwal/Desktop/pca.csv','w') as outfile:
    #    print((Xtra

In [18]:
from sklearn.decomposition import IncrementalPCA
import csv
import sys
import numpy as np
import pandas as pd
from pprint import pprint



with open('/Users/ravishaggarwal/Desktop/bigrams.csv','rt') as input_file, open('/Users/ravishaggarwal/Desktop/pca_transform_data.csv', 'w') as output_file:
    dataset = input_file
    chunksize_ = 10000
    dimensions = 1457
    reader = pd.read_csv(dataset, sep = ',', chunksize = chunksize_)
    sklearn_pca = IncrementalPCA(n_components=dimensions)
    for chunk in reader:
        sklearn_pca.partial_fit(chunk)
        print('chunking and printing')
        Xchunk = sklearn_pca.transform(chunk)
        a = Xchunk[:,:200]
        #pprint(a)
        b = np.asarray(a)
        #np.savetxt('/Users/ravishaggarwal/Desktop/pca_transform_data1.csv', a, delimiter=",")
        print(b.shape)
        csvWriter = csv.writer(output_file,delimiter=',')
        csvWriter.writerows(b)
        #for row in a:
            #b = float(row)
        #    print(row.shape)
        #    csvWriter.writerows(b)
        #print(*a,file = output_file)
        #for row in a:
            #for elem in row:
                #print(elem.shape)
                #print(elem, end=' ')#,file = output_file)

        #print(Xchunk.shape)
        #for s in Xchunk:
            #a = s[:100]
            #print(*a.shape)
            #print(*a,file = output_file)#,file = output_file)
        #print(Xchunk,file = output_file)

chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printin

(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunking and printing
(10000, 200)
chunkin

In [2]:
import dask.dataframe as dd 

In [3]:
df_dimred = dd.read_csv('/Users/ravishaggarwal/Desktop/pca_transform_data.csv')

In [28]:
len(df_dimred)

4675928

In [25]:
#df_dimred = pd.read_csv('/Users/ravishaggarwal/Desktop/pca_transform_data.csv')

In [26]:
#df_dimred.shape

(4675928, 200)

In [42]:
#elbow method 
#finding elbow method in amazon revie is difficult 
#X = df

#distortions = []

#K = range(1,5)
#for k in K:
#    kmeanModel = KMeans(n_clusters=k).fit(X)
#    kmeanModel.fit(X)
    #print(kmeanModel.inertia_)
#    distortions.append(kmeanModel.inertia_)

    
    
# Plot the elbow
#plt.plot(K, distortions, 'bx-')
#plt.xlabel('k')
#plt.ylabel('Distortion')
#plt.title('The Elbow Method showing the optimal k')
#plt.show()

In [5]:
import numpy as np

from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler


In [7]:
X = df_dimred

In [ ]:
db = DBSCAN(eps=0.05, min_samples=100).fit(X)

In [ ]:
#db = DBSCAN(eps=0.05, min_samples=100).fit(X)

In [9]:
from sklearn.cluster import KMeans
x = df
#num_clusters = 2
num_clusters = 3
kmeans = KMeans(n_clusters=num_clusters, max_iter = 100,n_init = 1)
kmeans.fit(x)

KeyboardInterrupt: 